In [91]:
import hashlib
from base64 import b64encode as b64
import sys, os, pathlib
import itertools

In [110]:
def fmt_hash(typ, meth, h):
    return "%s %s:%s" % (typ, meth, h)

def print_hash(typ, meth, h, f=None):
    s = fmt_hash(typ, meth, h)
    print(s, file=f or sys.stdout)
    
def print_hex(h, meth='', f=None):
    print_hash('hex', meth, h, f)
    
def print_HEX(h, meth='', f=None):
    print_hash('HEX', meth, h, f)
    
def print_b64(h, meth='', f=None):
    print_hash('base64', meth, h, f)

In [111]:
def bhH(d):
    b64s = b64(d)
    hexs = ''.join(hex(i)[2:] for i in d)
    HEXS = hexs.upper()
    return b64s, hexs, HEXS  

In [112]:
def calc_checksum_sha256(s):
    if not isinstance(s, bytes):
        s = s.encode('ascii')
    d = hashlib.sha256(s).digest()
    b, h, H = bhH(d)
    return d, b, h, H

def checksum_sha256(s):
    d, b, h, H = calc_checksum_sha256(s)
    m = 'sha256'
    print_hex(h, m)
    print_HEX(H, m)
    print_b64(b, m)
    return d, b, h, H

In [113]:
class UnknownMethod(ValueError):
    def __init__(self, msg):
        super().__init__("unsupported hash method: %s" % hash_method)

def checksum(ob, hash_method='sha256'):
    if isinstance(ob, (str)):
        ob = ob.encode('ascii')
    elif isinstance(ob, bytes):
        pass
    elif hasattr(ob, 'read'):
        ob = ob.read()
    else:
        raise TypeError(type(ob))
        
    if hash_method=='sha256':
        return checksum_sha256(ob)
    else:
        raise UnknownMethod(hash_method)

In [114]:
def pl2s(p):
    return str(p)

def get_bytes(ob):
    if isinstance(ob, bytes):
        return ob
    if isinstance(ob, str):
        return ob.encode('ascii')
    if isinstance(ob, pathlib.Path):
        with open(pl2s(ob), 'rb') as f:
            return f.read()
    if hasattr(ob, 'read'):
        return ob.read()
    raise TypeError(type(ob))
    
def checksum_file(file, method='sha256'):
    if isinstance(file, (str, pathlib.Path)):
        if isinstance(file, pathlib.Path):
            file = pl2s(file)
        with open(file, 'rb') as f:
            return checksum(f.read(), method)
    else:
        return checksum(file, method)

In [115]:
file = "E:\\BizHawk-1.11.6\\Firmware\\GBA.BIOS"
d=checksum_file(file)

hex sha256:fd2547724b505f487e6dcb29ec2ecff3af35a841a77ab2e85fd87350abd36570
HEX sha256:FD2547724B505F487E6DCB29EC2ECFF3AF35A841A77AB2E85FD87350ABD36570
base64 sha256:b'/SVHcktQX0h+bcsp7C7P8681qEGnerLoX9hzUKvTZXA='


In [123]:
def compare(h1, h2, method, typ):
    match = "MATCH"
    if h1 != h2:
        match = "MISMATCH"
    print(match)
    print_hash(typ, method, h1)
    print_hash(typ, method, h2)
    def putc(c): print(c, end="")
    if h1 != h2:
        f1, f2 = fmt_hash(typ, method, h1), fmt_hash(typ, method, h2)
        for a, b in itertools.zip_longest(f1, f2, fillvalue=" "):
            if a != b:
                putc("^")
            else:
                putc(" ")
            
                
    

def compare_checksum(ob, cs, cs_type='hex', method='sha256'):
    s = get_bytes(ob)
    if method=='sha256':
        d, b, h, H = calc_checksum_sha256(s)
    else:
        raise UnknownMethod(method)
    if cs_type == 'hex':
        compare(h, cs, method, cs_type)
    elif cs_type == 'HEX':
        compare(H, cs, method, cs_type)
    elif cs_type in ('b64', 'base64'):
        compare(b, cs, method, cs_type)
    else:
        raise UnknownMethod(cs_type)
        
        
            
    

In [124]:
def test():
    p = pathlib.Path(file)
    cs = "FD2547724B505F487E6DCB29EC2ECFF3AF35A841A77Aa2E85FD87350ABD36570"
    compare_checksum(p, cs, 'HEX')

In [125]:
def main():
    fp, cs = sys.argv[1:3]
    for typ in ('HEX', 'hex', 'base64', 'HEX'):
        if typ in sys.argv:
            break
    for meth in ("sha256",):
        if meth in sys.argv:
            break
    fp = pathlib.Path(fp)
    compare_checksum(fp, cs, typ, meth)

In [126]:
sys.argv[1:3] = [p, cs]
if __name__ == '__main__':
    main()

MISMATCH
HEX sha256:FD2547724B505F487E6DCB29EC2ECFF3AF35A841A77AB2E85FD87350ABD36570
HEX sha256:FD2547724B505F487E6DCB29EC2ECFF3AF35A841A77Aa2E85FD87350ABD36570
                                                       ^                   